# Exercise 4.1: Filters in CNNs

Advanced Machine Learning for KCS

by Stefan Heinrich, Stella Grasshof, Laura Weihl
with material by Kevin Murphy

All info and static material: https://learnit.itu.dk/course/view.php?id=3021295

-------------------------------------------------------------------------------

## Demonstration of some Filters as used in Convolutional Neural Networks

*Hint: We use pytorch for these practical examples. Read up about the concepts of tensor data structures and graph computation in the tutorials and the API that we have introduces last week, if needed!*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
# pyTorch, a powerful machine learning framework
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
# Helper method
def tensor_to_image(data: torch.Tensor):
  img = plt.imshow(data.numpy())
  img.set_cmap('gray')
  plt.axis('on')
  plt.xticks([])
  plt.yticks([])

### Convolution & filters

In [ ]:
def conv2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [ ]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
filtered = conv2d(X, K)

# plt.imshow(filtered.numpy(), cmap='gray')
tensor_to_image(filtered)

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
tensor_to_image(X)
print(pd.DataFrame(X.numpy()))
print(f" shape -> {X.shape}")

In [ ]:
kernel = torch.tensor([[1.0, -1.0], [1.0, -1.0]])
tensor_to_image(kernel)
print(pd.DataFrame(kernel.numpy()))
print(f" shape -> {kernel.shape}")

In [ ]:
x_filtered = conv2d(X, kernel)
tensor_to_image(x_filtered)
print(pd.DataFrame(x_filtered.numpy()))
print(f" shape -> {x_filtered.shape}")

In [ ]:
im = np.array(Image.open('bike.jpg').convert('L'))
im_tens = torch.Tensor(im)
tensor_to_image(im_tens)
print(f" shape -> {im_tens.shape}")


#### Apply our vertical and horizontal kernels and convolve them through the image.


In [ ]:
horizontal_res = conv2d(im_tens, kernel)
vertical_res = conv2d(im_tens, kernel.t())

In [ ]:
print(f"shape -> {vertical_res.shape}")
tensor_to_image(horizontal_res.abs())

In [ ]:
tensor_to_image(vertical_res.abs())
print(f"shape -> {vertical_res.shape}")

In [ ]:
c = (horizontal_res ** 2 + vertical_res ** 2) ** 0.5
tensor_to_image(c)
print(f"shape -> {c.shape}")

In [ ]:
sobel_hor = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_ver = sobel_hor.t()

In [ ]:
sob_engine_h = conv2d(im_tens, sobel_hor)
sob_engine_v = conv2d(im_tens, sobel_ver)

total_sobel_img = (sob_engine_h ** 2 + sob_engine_v ** 2) ** 0.5
tensor_to_image(total_sobel_img)

In [ ]:
# Sobel operator gradient directions
total_sobel_grad = torch.atan2(sob_engine_v, sob_engine_h)
tensor_to_image(total_sobel_grad)

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        # Kernel values
        self.weight = nn.Parameter(torch.rand(kernel_size))
        # Bias
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return conv2d(x, self.weight) + self.bias

In [ ]:
circle_im = torch.Tensor(np.array(Image.open('circle.jpg').convert('L')))
tensor_to_image(circle_im)

In [ ]:
filtered_circle = conv2d(circle_im, kernel)
tensor_to_image(filtered_circle.abs())

In [ ]:
filtered_circle_t = conv2d(circle_im, kernel.t())
tensor_to_image(filtered_circle_t.abs())

In [ ]:
tensor_to_image((filtered_circle_t ** 2 + filtered_circle ** 2) ** 0.5)

In [ ]:
conv_nn = Conv2D((2,2))

# Imput image from the previous example
X = torch.ones((6, 8))
X[:, 2:6] = 0

# Predesigned kernel from the previous example
Y = conv2d(X, kernel)

lr = 3e-2  # Learning rate

for i in range(300):
    Y_hat = conv_nn(X)
    l = (Y_hat - Y)**2
    conv_nn.zero_grad()
    l.mean().backward()
    # Update the kernel
    conv_nn.weight.data[:] -= lr * conv_nn.weight.grad
    if (i + 1) % 20 == 0:
        print(f'batch {i + 1}, loss {l.mean():.3f}')

In [ ]:
conv_nn.weight

### Padding

In [ ]:
circle_im = torch.Tensor(np.array(Image.open('circle.jpg').convert('L')))
print(f"shape -> {circle_im.shape}")
tensor_to_image(circle_im)

In [ ]:
padded_circle_im = F.pad(circle_im, (1,1,1,1), value=1)
print(f"shape -> {padded_circle_im.shape}")
tensor_to_image(padded_circle_im)

In [ ]:
f_circle_im = conv2d(padded_circle_im, sobel_ver)
print(f"shape -> {f_circle_im.shape}")
tensor_to_image(f_circle_im.abs())